In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-pr/sample_submission.csv
/kaggle/input/santander-pr/train.csv
/kaggle/input/santander-pr/test.csv


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
data_new = pd.read_csv('/kaggle/input/santander-pr/train.csv', dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                  'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                  'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                  'ind_nomina_ult1':"float16", 'ind_nom_pens_ult1':"float16",
                                                                  'ind_actividad_cliente':"float16", 'cod_prov':"float16",
                                                                  'tipodom':"float16", "indrel":"float16", "ind_nuevo":"float16",
                                                                  'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                  'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                  'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                  'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"},
                      parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'])
data_test = pd.read_csv('/kaggle/input/santander-pr/test.csv', parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_new['Year_Month'] = data_new['fecha_dato'].apply(lambda x: (100 * x.year) + x.month)
data_new.set_index('Year_Month', inplace = True)

In [5]:
my_list = ["201507", "201604"]
data_new = data_new[data_new.index.isin(my_list)]

In [6]:
print(data_new['ncodpers'].nunique())
print(data_test['ncodpers'].nunique())

938136
931453


In [7]:
data_new.drop(columns = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_deco_fin_ult1'], inplace = True)

In [8]:
lag = [1]
data_new = pd.concat([data_new]+[data_new.groupby('ncodpers', as_index = False)[
                'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
                   'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
                   'ind_ctpp_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
                   'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
                   'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
                   'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
                   'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'].shift(-1).add_suffix('_lag'+ str(x)) for x in lag],axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


In [9]:
data_merged = pd.concat([data_test.assign(ind="test"), data_new.assign(ind="train")])

In [10]:
del data_new
del data_test

In [11]:
data_merged.drop(columns = ['ult_fec_cli_1t', 'tipodom', 'indext', 'tiprel_1mes', 'nomprov'], inplace = True)

In [12]:
# Replacing cod_prov null values with Madrid's cod_prov as Madrid is the most occuring nomprov
replace_with_mode_cols = ['cod_prov', 'canal_entrada', 'conyuemp', 'sexo', 'indrel_1mes', 'segmento']
data_merged[replace_with_mode_cols]=data_merged[replace_with_mode_cols].fillna(data_merged.mode().iloc[0])
# for col in range(len(replace_with_mode_cols)) :
#     data_merged[replace_with_mode_cols[col]].fillna(data_merged[replace_with_mode_cols[col]].mode()[0], inplace=True)

renta_vs_cod = data_merged[['age', 'renta', 'cod_prov', 'segmento']][data_merged['renta'].notnull()]
replacement = renta_vs_cod.groupby(by=['segmento', 'cod_prov'],  sort=True, as_index = False, dropna = False).mean() # not null ones
replacement = pd.DataFrame(replacement, columns = ['segmento', 'cod_prov', 'renta'])

#creating a dictionary for replacing null values of renta with help of cod_prov and segmento
dict = pd.Series(replacement['renta'].values,index=replacement['cod_prov'].astype(str) + '#' + replacement['segmento']).to_dict()

renta_mean = data_merged['renta'].mean()

def replaceRenta(key) :
    if key in dict :
        return dict.get(key)
    else :
        return renta_mean

In [13]:
data_merged.loc[data_merged['renta'].isna(), 'renta'] = data_merged[data_merged['renta'].isna()].apply(lambda x : replaceRenta(str(x['cod_prov']) + '#' + str(x['segmento'])), axis=1)

In [14]:
import datetime as dt
data_merged['fecha_dato']=data_merged['fecha_dato'].map(dt.datetime.toordinal)
data_merged['fecha_alta']=data_merged['fecha_alta'].map(dt.datetime.toordinal)

In [15]:
ce_random = [241, 213, 243, 228, 109, 111, 387, 290, 28, 255, 446, 374, 105, 466, 438, 103, 403, 88, 492, 140, 254, 211, 427, 216, 29, 412, 114, 33, 302, 100, 481, 272, 91, 451, 163, 335, 193, 471, 99, 160, 425, 218, 19, 169, 326, 148, 314, 341, 445, 237, 373, 183, 16, 153, 239, 154, 271, 414, 21, 199, 324, 41, 296, 459, 76, 497, 104, 87, 424, 417, 279, 369, 423, 129, 487, 280, 179, 408, 443, 450, 263, 77, 276, 121, 110, 420, 460, 156, 437, 190, 281, 496, 340, 317, 184, 83, 389, 32, 200, 166, 171, 375, 258, 18, 448, 146, 209, 467, 72, 409, 396, 231, 44, 390, 273, 323, 444, 364, 39, 63, 268, 43, 353, 278, 485, 47, 318, 305, 270, 362, 196, 38, 46, 257, 244, 349, 6, 123, 35, 315, 311, 245, 236, 145, 490, 180, 208, 261, 192, 407, 301, 411, 312, 269, 413, 57, 300, 422, 80, 3, 700, 701]
ps_random = [680, 726, 658, 734, 746, 666, 604, 628, 749, 742, 762, 667, 792, 751, 738, 627, 744, 784, 618, 707, 689, 723, 635, 780, 754, 703, 783, 729, 700, 624, 650, 773, 763, 715, 632, 659, 717, 652, 702, 681, 740, 654, 692, 709, 670, 664, 708, 613, 794, 771, 752, 769, 779, 755, 600, 611, 705, 660, 781, 758, 691, 759, 790, 732, 788, 631, 623, 725, 638, 776, 640, 718, 747, 686, 774, 698, 663, 610, 687, 713, 642, 648, 739, 639, 770, 714, 782, 750, 672, 605, 620, 641, 608, 656, 785, 696, 603, 745, 673, 777, 761, 716, 647, 615, 753, 668, 633, 693, 719, 786, 791, 797, 736, 796, 712, 609, 737, 765]

In [16]:
unique_canal_values = list(data_merged['canal_entrada'].unique())
unique_ps_values = list(data_merged['pais_residencia'].unique())
ce_dict = {unique_canal_values[i]: ce_random[i] for i in range(len(unique_canal_values))}
ps_dict = {unique_ps_values[i]: ps_random[i] for i in range(len(unique_ps_values))}
data_merged["canal_entrada"] = data_merged["canal_entrada"].map(ce_dict)
data_merged["pais_residencia"] = data_merged["pais_residencia"].map(ps_dict)
data_merged['antiguedad'] = pd.to_numeric(data_merged['antiguedad'], errors='coerce')
data_merged['age'] = pd.to_numeric(data_merged['age'], errors='coerce')

In [17]:
encoder_dict = {
'sexo'          : {'V':0, 'H':1, -99:2},
'indrel_1mes'   : {-99:0, '1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:3},
'conyuemp'      : {-99:0, 'N' : 11, 'S' : 13},
'indfall'       : {'N' : 6, 'S' : 1},
'ind_empleado'  : {'N' : 3, 'B': 7, 'F': 9, 'A': 20, 'S': 13}
}
data_merged.replace(encoder_dict, inplace=True)

In [18]:
data_test, data_train = data_merged[data_merged["ind"].eq("test")], data_merged[data_merged["ind"].eq("train")]
data_test.drop(columns = ['ind'], inplace = True)
data_train.drop(columns = ['ind'], inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
del data_merged

In [20]:
feature_cols = ['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'indrel_1mes', 'indresi',
       'conyuemp', 'canal_entrada', 'indfall', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_cco_fin_ult1_lag1', 'ind_cder_fin_ult1_lag1',
       'ind_cno_fin_ult1_lag1', 'ind_ctju_fin_ult1_lag1',
       'ind_ctma_fin_ult1_lag1', 'ind_ctop_fin_ult1_lag1',
       'ind_ctpp_fin_ult1_lag1', 'ind_deme_fin_ult1_lag1',
       'ind_dela_fin_ult1_lag1', 'ind_ecue_fin_ult1_lag1',
       'ind_fond_fin_ult1_lag1', 'ind_hip_fin_ult1_lag1',
       'ind_plan_fin_ult1_lag1', 'ind_pres_fin_ult1_lag1',
       'ind_reca_fin_ult1_lag1', 'ind_tjcr_fin_ult1_lag1',
       'ind_valo_fin_ult1_lag1', 'ind_viv_fin_ult1_lag1',
       'ind_nomina_ult1_lag1', 'ind_nom_pens_ult1_lag1',
       'ind_recibo_ult1_lag1']
product_cols = ['ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
print(len(product_cols))

21


In [21]:
data_test.drop(columns = product_cols, inplace = True)

In [22]:
lag_columns = data_train.columns[data_train.isna().any()].tolist()

In [23]:
for col in range(len(lag_columns)) :
     data_train[lag_columns[col]].fillna(value = 0, inplace=True)     
     data_test[lag_columns[col]].fillna(value = 0, inplace=True)

In [ ]:
# X = data_train.drop(columns = product_cols, inplace = False)
# y = data_train.drop(columns = feature_cols, inplace = False)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# model= RandomForestClassifier(random_state=1, n_jobs = -1, verbose = 2, n_estimators = 100, max_depth = 5)
# model.fit(X, y)

In [ ]:
# for i, j in sorted(zip(X.columns, model.feature_importances_)):
#     print(i, j)

In [24]:
drop_columns = ['conyuemp', 'fecha_dato', 'ind_empleado', 'indfall', 'indrel_1mes', 'indresi']
data_train.drop(columns = drop_columns, inplace = True)
data_test.drop(columns = drop_columns, inplace = True)

In [25]:
feature_cols = ['ncodpers', 'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo',
       'antiguedad', 'indrel', 'canal_entrada', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento','ind_cco_fin_ult1_lag1',
       'ind_cder_fin_ult1_lag1', 'ind_cno_fin_ult1_lag1',
       'ind_ctju_fin_ult1_lag1', 'ind_ctma_fin_ult1_lag1',
       'ind_ctop_fin_ult1_lag1', 'ind_ctpp_fin_ult1_lag1',
       'ind_deme_fin_ult1_lag1', 'ind_dela_fin_ult1_lag1',
       'ind_ecue_fin_ult1_lag1', 'ind_fond_fin_ult1_lag1',
       'ind_hip_fin_ult1_lag1', 'ind_plan_fin_ult1_lag1',
       'ind_pres_fin_ult1_lag1', 'ind_reca_fin_ult1_lag1',
       'ind_tjcr_fin_ult1_lag1', 'ind_valo_fin_ult1_lag1',
       'ind_viv_fin_ult1_lag1', 'ind_nomina_ult1_lag1',
       'ind_nom_pens_ult1_lag1', 'ind_recibo_ult1_lag1']
product_cols = ['ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1']
print(len(product_cols))

21


In [26]:
X = data_train.drop(columns = product_cols, inplace = False)
y = data_train.drop(columns = feature_cols, inplace = False)

In [27]:
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = data_test['ncodpers'].values
for c in product_cols:
    if c != 'ncodpers':
        print(c)
        
#         y_train = y[c]
#         x_train = X
        
        clf = LogisticRegression(random_state = 42, verbose = 1, n_jobs = -1, warm_start = True)

        clf.fit(X, y[c])
        p_train0 = clf.predict_proba(data_test)[:,0]
        p_train1 = clf.predict_proba(data_test)[:,1]

        # pridict model with the most recent data
        #y_train2 = df_train[c]
#         x_train2 = data_test.drop(['ncodpers',"fecha_dato"], 1)
#         p_train = clf.predict_proba(x_train2)[:,1]
#         p_train2 = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train1
        
        for id, p in zip(ids, p_train1):
            id_preds[id].append(p)

ind_cco_fin_ult1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   30.4s finished


ind_cder_fin_ult1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.9min finished


ind_cno_fin_ult1


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   41.1s finished


ind_nom_pens_ult1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   57.5s finished


ind_recibo_ult1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   36.0s finished


In [32]:
data_train.columns

Index(['ncodpers', 'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo',
       'antiguedad', 'indrel', 'canal_entrada', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1', 'ind_cco_fin_ult1_lag1',
       'ind_cder_fin_ult1_lag1', 'ind_cno_fin_ult1_lag1',
       'ind_ctju_fin_ult1_lag1', 'ind_ctma_fin_ult1_lag1',
       'ind_ctop_fin_ult1_lag1', 'ind_ctpp_fin_ult1_lag1',
       'ind_deme_fin_ult1_lag1', 'ind_dela_fin_ult1_lag1',
       'ind_ecue_fin_ult1_lag1', 'ind_fond_fin_ult1_lag1',
      

In [35]:
feature_cols = ['pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo',
       'antiguedad', 'indrel', 'canal_entrada', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_cco_fin_ult1_lag1',
       'ind_cder_fin_ult1_lag1', 'ind_cno_fin_ult1_lag1',
       'ind_ctju_fin_ult1_lag1', 'ind_ctma_fin_ult1_lag1',
       'ind_ctop_fin_ult1_lag1', 'ind_ctpp_fin_ult1_lag1',
       'ind_deme_fin_ult1_lag1', 'ind_dela_fin_ult1_lag1',
       'ind_ecue_fin_ult1_lag1', 'ind_fond_fin_ult1_lag1',
       'ind_hip_fin_ult1_lag1', 'ind_plan_fin_ult1_lag1',
       'ind_pres_fin_ult1_lag1', 'ind_reca_fin_ult1_lag1',
       'ind_tjcr_fin_ult1_lag1', 'ind_valo_fin_ult1_lag1',
       'ind_viv_fin_ult1_lag1', 'ind_nomina_ult1_lag1',
       'ind_nom_pens_ult1_lag1', 'ind_recibo_ult1_lag1']
data_train.drop(columns = feature_cols, inplace = True)

In [36]:
product_cols = ['ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1']
len(product_cols)

21

In [38]:
data_train.shape

(1758091, 22)

In [ ]:
customer_prediction_label

In [40]:
already_active = {}
already_inactive = {}
for row in data_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(tuple(product_cols), row) if c[1] > 0] # galti is here
    inactive = [c[0] for c in zip(tuple(product_cols), row) if c[1] == 0]
    already_active[id] = active
    already_inactive[id] = inactive

In [42]:
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
#     print(id)
#     print(already_active.get(id,[]))
#     print(already_inactive.get(id,[]))
    added_preds = [i for i in sorted([i for i in zip(tuple(product_cols), p) if i[0] not in already_active.get(id, [])],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]]
    discontinued_preds = [i for i in sorted([i for i in zip(tuple(product_cols), p) if i[0] not in already_inactive.get(id, [])],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]]
    total_preds = added_preds + discontinued_preds
    total_preds.sort(key=lambda x: x[1], reverse = True)
    total_preds = [x[0] for x in total_preds][:5]
    train_preds[id] = total_preds  

In [50]:
test_preds = []
for row in data_test.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [51]:
submission = pd.DataFrame({'ncodpers':data_test['ncodpers'],'changed':test_preds})
submission.to_csv('LogisticRegressionWithLag_AddDisc3.csv', index=False)

In [ ]:
submission